In [1]:
import cv2
import numpy as np

def nothing(x):
    pass

cap = cv2.VideoCapture(0)#♦video çağırma
cv2.namedWindow("Settings")#pencere
#track bar oluşturma ve hareket ettirme
cv2.createTrackbar("Lower-Hue","Settings",0,180,nothing)#kızak, adı pencere adı, değerleri, 
cv2.createTrackbar("Lower-Saturation","Settings",0,255,nothing)
cv2.createTrackbar("Lower-Value","Settings",0,255,nothing)
cv2.createTrackbar("Upper-Hue","Settings",0,180,nothing)
cv2.createTrackbar("Upper-Saturation","Settings",0,255,nothing)
cv2.createTrackbar("Upper-Value","Settings",0,255,nothing)

font = cv2.FONT_HERSHEY_SIMPLEX #şekillerin adını yazarken kullanılan font 

#tek tek frameleri çekip tahmin edicez
while 1:
    ret,frame = cap.read()
    frame = cv2.flip(frame,1)#frameleri ters dondur
    hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)#frameleri hsv formatına cevir
    
    #kızakları alıp ilk ve son değerlerini lower upper e kaydediyorum
    lh = cv2.getTrackbarPos("Lower-Hue","Settings")
    ls = cv2.getTrackbarPos("Lower-Saturation","Settings")
    lv = cv2.getTrackbarPos("Lower-Value","Settings")
    uh = cv2.getTrackbarPos("Upper-Hue","Settings")
    us = cv2.getTrackbarPos("Upper-Saturation","Settings")
    uv = cv2.getTrackbarPos("Upper-Value","Settings")
    
    #aldığım değerlerı saklamak için lower color ve upper color oluşturup saklayacağız
    lower_color = np.array([lh,ls,lv])
    upper_color = np.array([uh,us,uv])
    
    #mask değişkeni oluşturarak o değerleri uygulayacağım
    mask = cv2.inRange(hsv,lower_color,upper_color)
    kernel = np.ones((5,5),np.uint8)#beyaz nesneler üzerinde oluşan siyah noktaları yok etmek için kernel uygulayacağız
    mask = cv2.erode(mask,kernel)#kernel uygula

    contours,_ = cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)#maskta konturları arayabiliriz

    #cizimleri yapmak için cnt olusturucam
    for cnt in contours:
        
        area  = cv2.contourArea(cnt)#kontur alan hesabı
        epsilon = 0.02*cv2.arcLength(cnt,True)
        approx = cv2.approxPolyDP(cnt,epsilon,True)#cnt ile bulduğum konturlara iyilestirme yapıcam
        
        #konturların basladığı ilk noktalar
        x = approx.ravel()[0]
        y = approx.ravel()[1]
        
        #bulduğun cokgenin alanı 400 den buyukse goster
        if area > 400:
            cv2.drawContours(frame,[approx],0,(0,0,0),5)
            
            if len(approx)==3:#ucgense
                cv2.putText(frame,"Triangle",(x,y),font,1,(0,0,0))
                
            elif len(approx)==4:#dortgense
                cv2.putText(frame,"Rectangle",(x,y),font,1,(0,0,0))
                
            elif len(approx)>6:#besgense
                cv2.putText(frame,"Circle",(x,y),font,1,(0,0,0))

    cv2.imshow("frame",frame)
    cv2.imshow("mask",mask)

    if cv2.waitKey(3) & 0xFF == ord('q'):#q ya basınca çıkma
        break

cap.release()
cv2.destroyAllWindows()
